In [4]:
%run ../../src/feature_selection_scripts/Feature_Selection_Bin_Cla_SRV.ipynb

['Seq', 'sTos', 'sTtl', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'AckDat', 'icmp', 'tcp', 'udp', 'ECO', 'FIN', 'INT', 'RST', 'Start']
(911917, 15)


In [5]:
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y_attack_type_train_encoded = encoder.fit_transform(y_attack_type_train)
y_attack_type_train_encoded = to_categorical(y_attack_type_train_encoded)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

num_classes = y_attack_type_train_encoded.shape[1]
num_features = X_train.shape[1]
sequence_length = 1



In [8]:
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [9]:
model = Sequential([
    LSTM(50, input_shape=(1, X_train.shape[1])),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Attempt to train again
model.fit(X_train_reshaped, y_attack_type_train_encoded, epochs=20, batch_size=32)

Epoch 1/20


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


28498/28498 ━━━━━━━━━━━━━━━━━━━━ 18s 603us/step - accuracy: 0.9368 - loss: 0.2202
Epoch 2/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 603us/step - accuracy: 0.9896 - loss: 0.0403
Epoch 3/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 18s 627us/step - accuracy: 0.9925 - loss: 0.0273
Epoch 4/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 20s 708us/step - accuracy: 0.9937 - loss: 0.0219
Epoch 5/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 611us/step - accuracy: 0.9944 - loss: 0.0188
Epoch 6/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 583us/step - accuracy: 0.9949 - loss: 0.0172
Epoch 7/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 18s 628us/step - accuracy: 0.9955 - loss: 0.0152
Epoch 8/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 18s 621us/step - accuracy: 0.9959 - loss: 0.0142
Epoch 9/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 600us/step - accuracy: 0.9960 - loss: 0.0132
Epoch 10/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 608us/step - accuracy: 0.9963 - loss: 0.0121
Epoch 11/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 18s 633us/step - accuracy: 0.9964 -

In [10]:
import pickle
with open('../VotingClassifier/voting_clf.pkl', 'rb') as f:
    voting_clf = pickle.load(f)

In [11]:
y_pred_binary = voting_clf.predict(X_test)

In [12]:
X_malicious_test = X_test[y_pred_binary == 1]

y_attack_type_test_malicious = y_attack_type_test[y_pred_binary == 1]
y_attack_type_test_malicious_encoded = encoder.transform(y_attack_type_test_malicious)
y_attack_type_test_malicious_encoded = to_categorical(y_attack_type_test_malicious_encoded, num_classes=num_classes)

In [13]:
X_malicious_test_reshaped = X_malicious_test.reshape((X_malicious_test.shape[0], 1, X_malicious_test.shape[1]))

In [14]:
y_pred_attack_type = model.predict(X_malicious_test_reshaped)

_, accuracy = model.evaluate(X_malicious_test_reshaped, y_attack_type_test_malicious_encoded)
print(f'Model accuracy on malicious test data: {accuracy*100:.2f}%')

5755/5755 ━━━━━━━━━━━━━━━━━━━━ 2s 306us/step
5755/5755 ━━━━━━━━━━━━━━━━━━━━ 2s 325us/step - accuracy: 0.9969 - loss: 0.0089
Model accuracy on malicious test data: 99.69%


In [20]:
import keras
keras.saving.save_model(model, 'LSTM_model.keras')